In [ ]:
!pip install pyngrok

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 14.9 MB/s eta 0:00:00


In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 35.3 MB/s eta 0:00:00


In [ ]:
# Importing the drive module from Google Colab to mount Google Drive (Authorization).
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, request, render_template, jsonify  # Import Flask and other modules for web application
from flask import Flask, send_file  # Import Flask and send_file for serving images
from io import BytesIO  # Import BytesIO for handling binary data
import matplotlib.pyplot as plt  # Import matplotlib for plotting images
from PIL import Image  # Import PIL for image processing
import matplotlib.image as mpimg  # Import matplotlib.image for reading images
from ultralytics import YOLO # YOLO object detection model
import mediapipe as mp # Mediapipe for pose estimation
import cv2 # OpenCV for image processing
import os # For file operations
import random # For random number generation
from matplotlib.patches import Rectangle  # For drawing rectangles on images

import math  # For mathematical operations
import io  # For handling binary data
import base64  # For encoding/decoding binary data
import numpy as np  # For numerical operations
from operator import imod  # For modulo operation

from pyngrok import ngrok  # For exposing local server to the internet


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load a trained model
model = YOLO('/content/drive/MyDrive/Hassana/Run/runs/detect/train/weights/best.pt')

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token("2oq6itLuYMRRLskGyq4t7jnTwkw_3FGnQttFhNCNYQuzCrTFb")
public_url =  ngrok.connect(5000).public_url
print(public_url)

# Load a trained model
model = YOLO('/content/drive/MyDrive/Hassana/Run/runs/detect/train/weights/best.pt')

@app.route('/Ruku', methods=['POST', 'GET'])
def Detect_Ruku():
    imagefile = request.files['imagefile']
    image_path = imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()


    # Function to calculate angle between lines formed by three points
    def calculate_angle(a, b, c):
        angle_rad1 = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
        angle_rad1 = angle_rad1 % (2 * math.pi)
        angle_rad2 = math.atan2(a[1]-b[1], a[0]-b[0]) - math.atan2(c[1]-b[1], c[0]-b[0])
        angle_rad2 = angle_rad2 % (2 * math.pi)
        return math.degrees(min(angle_rad1, angle_rad2))

    # Function to check if the pose resembles the ruku position in Salat
    def is_ruku_pose(landmarks):
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
        right_hip = (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
        right_shoulder = (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y)
        right_knee = (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)

        angle_left = abs(90 - calculate_angle(left_knee, left_hip, left_shoulder))
        angle_right = abs(90 - calculate_angle(right_knee, right_hip, right_shoulder))

        ruku_angle_threshold = 30
        if angle_left <= ruku_angle_threshold or angle_right <= ruku_angle_threshold:
            return True, min(abs(angle_left), abs(angle_right)) / 100 * 50 # Calculate error percentage
        else:
            return False, min(abs(angle_left), abs(angle_right)) / 100 * 50

    def process_image(image_path):
        results = model.predict(image_path)

        for r in results:
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()

        bounding_box = tuple(int(value) for value in bounding_box)

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        is_ruku, error_percentage = is_ruku_pose(results.pose_landmarks.landmark)
        print(error_percentage)

        # Red color if error>15, else green

        if error_percentage<= 15:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3

            # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)


        Perfect_Threshold = 5;


        plt.figure(figsize=(8, 8))
        plt.imshow(image_rgb)
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#onvert these normalized coordinates to pixel coordinates, we need to scale them according to the size of the actual image in pixels
        for landmark in [mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
                        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
            plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                        results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                        color='white', s=100)

        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].y * image.shape[0]],
                color='cyan', linestyle='-', linewidth=2, alpha=0.5)
        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * image.shape[0]],
                color='cyan', linestyle='-', linewidth=2, alpha=0.5)

        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image.shape[0]],
                color='magenta', linestyle='-', linewidth=2, alpha=0.5)
        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image.shape[0]],
                color='magenta', linestyle='-', linewidth=3, alpha=0.5)

        print('Done')
        plt.axis('off')
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()
        return image_base64, error_percentage, is_ruku



    # Process the image and get the base64 encoded plot
    plot_base64, error, is_ruku= process_image(image_path)
    error_text = "{:.2f}".format(error)

    if is_ruku and error <= 20:
        return jsonify({'image_base64': plot_base64, 'is_pose': True})
    else:
        return jsonify({'image_base64': plot_base64, 'is_pose': False})


@app.route('/Takbeer', methods=['POST', 'GET'])
def Detect_Takbeer():
    imagefile = request.files['imagefile']
    image_path =  imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Function to calculate angle between lines formed by three points
    def calculate_angle(a, b, c):
        angle_rad = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
        angle_rad = angle_rad % (2 * math.pi)
        return math.degrees(angle_rad)

    # Function to check if the pose resembles the Takbeer position in Salat
    def is_takbeer_pose(landmarks):
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
        right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        print(abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.22 * 100)

        Perfect_Threshold = 5;
        # Check if left and right wrist are on the same line
        if abs(left_wrist.y - right_wrist.y) < 0.05 and abs(left_wrist.x - right_wrist.x) > 0.15:
            # Check if distance between right hip and right shoulder in y-direction is big enough
            if right_hip.y - right_shoulder.y > 0.25 or left_hip.y - left_shoulder.y > 0.25 :
              return True, abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.25 * 100
        return False, abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.25 * 100

    def process_image(image_path):
        model_results = model.predict(image_path)

        for r in model_results:
            print(r.boxes)
            # Extract bounding box coordinates
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
            print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
            bounding_box_tuple = tuple(bounding_box)

        bounding_box = tuple(int(value) for value in bounding_box_tuple)

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Detect pose landmarks
        results = pose.process(image_rgb)
        is_takbeer, error = is_takbeer_pose(results.pose_landmarks.landmark)
        print(error)

        # Red color if error>15, else green

        if error<= 15 and is_takbeer:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3


        # Load image
        # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)

        # Convert image to RGB

        Perfect_Threshold = 5;



        # Draw text above bounding box with background color
        # Draw text above bounding box with background color
        plt.figure(figsize=(8, 8))
        plt.imshow(image_rgb)

        # Draw connections between all specified landmarks
        landmarks = results.pose_landmarks.landmark
        connections = [(mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                      (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
                      (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
                      (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
                      (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_WRIST),
                      (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_WRIST),
                      (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX),
                      (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_INDEX)]

        # Draw points for knees, hips, and shoulders
        for landmark in [mp_pose.PoseLandmark.LEFT_INDEX, mp_pose.PoseLandmark.RIGHT_INDEX,
                        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
                            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
                plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                            results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                            color='white', s=100)

        for connection in connections:
            start_point = connection[0]
            end_point = connection[1]
            plt.plot([landmarks[start_point.value].x * image.shape[1], landmarks[end_point.value].x * image.shape[1]],
                    [landmarks[start_point.value].y * image.shape[0], landmarks[end_point.value].y * image.shape[0]],
                    color='magenta', linestyle='-', linewidth=1)
        # Draw landmarks and connections
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=5),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))
        plt.axis('off')
        # Save the plot as bytes in memory
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        # Convert the plot bytes to base64 string
        plot_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()  # Close the plot to free memory

        return plot_base64, error, is_takbeer

    # Process the image and get the base64 encoded plot
    plot_base64,error, is_takbeer = process_image(image_path)

    error_text = "{:.2f}".format(error)


    if is_takbeer and error <= 20:
        return jsonify({'image_base64': plot_base64, 'is_pose': True})
    else:
        return jsonify({'image_base64': plot_base64, 'is_pose': False})



@app.route('/Raising', methods=['POST', 'GET'])
def Detect_Raising():
    imagefile = request.files['imagefile']
    image_path = imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    def is_raising_pose(landmarks):
        # Extract relevant landmarks
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
        chin = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

        print(abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100)
        print(left_wrist.x - right_wrist.x)
        print(left_wrist.y - right_wrist.y)

        # Check if left and right wrist are close to each other
        if abs(left_wrist.x - right_wrist.x) and abs(left_wrist.y - right_wrist.y) < 0.05:
            # Check if distance between right hip and right shoulder in y-direction is big enough
            if right_hip.y - right_shoulder.y > 0.25 or left_hip.y - left_shoulder.y > 0.25 :
                if right_wrist.y - right_shoulder.y > 0.15 or left_wrist.y - left_shoulder.y > 0.15:
                    # Check  wrists are above the shoulders
                    return True, abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100
        return False, abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100

    def process_image(image_path):
        # Load Model
        results = model.predict(image_path)

        for r in results:
            print(r.boxes)
            # Extract bounding box coordinates
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
            print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
            bounding_box_tuple = tuple(bounding_box)

        bounding_box = tuple(int(value) for value in bounding_box_tuple)

        # Load image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Detect pose landmarks
        results = pose.process(image_rgb)
        is_raising, error = is_raising_pose(results.pose_landmarks.landmark)
        print(error)



        # Red color if error>15, else green

        if error<= 15 and is_raising:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3


        # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)

        # Convert image to RGB

        Perfect_Threshold = 5;



        # Check if pose resembles raising pose

        # Draw text above bounding box with background color
        plt.figure(figsize=(8, 8))
        plt.savefig(image_path)  # Specify the file name and extension
        plt.imshow(image_rgb)

        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Draw connections between all specified landmarks
        landmarks = results.pose_landmarks.landmark
        connections = [(mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
                    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
                    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
                    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_WRIST),
                    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_WRIST),
                    (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX),
                    (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_INDEX)]

        # Draw points for knees, hips, and shoulders
        for landmark in [mp_pose.PoseLandmark.LEFT_INDEX, mp_pose.PoseLandmark.RIGHT_INDEX,
                        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
                            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
                plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                            results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                            color='white', s=100)

        for connection in connections:
            start_point = connection[0]
            end_point = connection[1]
            plt.plot([landmarks[start_point.value].x * image.shape[1], landmarks[end_point.value].x * image.shape[1]],
                    [landmarks[start_point.value].y * image.shape[0], landmarks[end_point.value].y * image.shape[0]],
                    color='magenta', linestyle='-', linewidth=1)
        # Draw landmarks and connections
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=5),
                                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))

        plt.axis('off')
        # Save the plot as bytes in memory
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        # Convert the plot bytes to base64 string
        plot_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

        plt.close()  # Close the plot to free memory

        return plot_base64, error, is_raising

    # Process the image and get the base64 encoded plot
    plot_base64, error, is_raising = process_image(image_path)
    error_text = "{:.2f}".format(error)


    if is_raising and error <= 20:
        return jsonify({'image_base64': plot_base64, 'is_pose': True})
    else:
        return jsonify({'image_base64': plot_base64, 'is_pose': False})


@app.route('/Sujud', methods=['POST', 'GET'])
def Detect_Sujud():
    imagefile = request.files['imagefile']
    image_path = imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)

    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)

    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)
    print('The predicted class is ' + str(bounding_box_int))

    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Function to calculate angle between lines formed by three points
    def calculate_angle(a, b, c):
        angle_rad1 = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
        angle_rad1 = angle_rad1 % (2 * math.pi)
        angle_rad2 = math.atan2(a[1]-b[1], a[0]-b[0]) - math.atan2(c[1]-b[1], c[0]-b[0])
        angle_rad2 = angle_rad2 % (2 * math.pi)
        return math.degrees(min(angle_rad1, angle_rad2))

    # Function to check if the pose resembles the Sujud position in Salat
    def is_sujud_pose(landmarks):
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

        # 1. Knees should be below hips (indicating kneeling)
        if left_knee.y > left_hip.y and right_knee.y > right_hip.y:
            # 2. Check if wrists are close to the nose's vertical position
            if abs(left_wrist.y - nose.y) < 0.1 and abs(right_wrist.y - nose.y) < 0.1:
                # 3. Nose should be lower than hips (indicating the head is close to the ground)
                if nose.y > left_hip.y and nose.y > right_hip.y:
                    return True, abs(left_wrist.y - nose.y) * 2 * 100
        return False, 0

    # Function to process image and detect Sujud pose
    def process_image(image_path):
        model_results = model.predict(image_path)

        for r in model_results:
            # Extract bounding box coordinates
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
            bounding_box_tuple = tuple(bounding_box)

        bounding_box = tuple(int(value) for value in bounding_box_tuple)

        # Load image
        image = cv2.imread(image_path)

        # Convert image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect pose landmarks
        results = pose.process(image_rgb)

        # Check if pose resembles Sujud pose
        is_sujud, error = is_sujud_pose(results.pose_landmarks.landmark)

        # Red color if error>15, else green
        if is_sujud and error <= 15:
            color = (0, 255, 0)
        else:
            color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3

        # Draw bounding box with the determined color
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)


        # Convert image to RGB

        Perfect_Threshold = 5;

        # Draw landmarks and connections
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=5),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))

        buffer = io.BytesIO()
        plt.imshow(image_rgb)
        plt.axis('off')
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()
        return image_base64, error, is_sujud

    # Process the image and get the base64 encoded plot
    plot_base64, error, is_sujud = process_image(image_path)
    error_text = "{:.2f}".format(error)

    if is_sujud and error <= 20:
        return jsonify({'image_base64': plot_base64, 'is_pose': True})
    else:
        return jsonify({'image_base64': plot_base64, 'is_pose': False})


if __name__ == "__main__":
    app.run(port=5000)


https://ad23-34-169-97-50.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



image 1/1 /content/1000000074.jpg: 448x640 1 raising, 64.8ms
Speed: 16.6ms preprocess, 64.8ms inference, 1172.1ms postprocess per image at shape (1, 3, 448, 640)
Bounding box coordinates (x1, y1, x2, y2): 805.905517578125 223.730224609375 1770.93115234375 1696.0
The predicted class is (805, 223, 1770, 1696)

image 1/1 /content/1000000074.jpg: 448x640 1 raising, 8.7ms
Speed: 3.8ms preprocess, 8.7ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
40.42092565025
Done


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:00:02] "POST /Ruku HTTP/1.1" 200 -



image 1/1 /content/1000000074.jpg: 448x640 1 raising, 35.3ms
Speed: 2.4ms preprocess, 35.3ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
Bounding box coordinates (x1, y1, x2, y2): 805.905517578125 223.730224609375 1770.93115234375 1696.0
The predicted class is (805, 223, 1770, 1696)

image 1/1 /content/1000000074.jpg: 448x640 1 raising, 9.1ms
Speed: 3.7ms preprocess, 9.1ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9424], device='cuda:0')
data: tensor([[8.0591e+02, 2.2373e+02, 1.7709e+03, 1.6960e+03, 9.4242e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1696, 2560)
shape: torch.Size([1, 6])
xywh: tensor([[1288.4183,  959.8651,  965.0256, 1472.2698]], device='cuda:0')
xywhn: tensor([[0.5033, 0.5660, 0.3770, 0.8681]], device='cuda:0')
xyxy: tensor([[ 805.9055,  223.7302, 1770.9312, 1696.0000]], device='cuda:0')

...

INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:09:25] "POST /Ruku HTTP/1.1" 200 -



image 1/1 /content/1000000089.jpg: 640x480 1 ruku, 34.8ms
Speed: 2.8ms preprocess, 34.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
Bounding box coordinates (x1, y1, x2, y2): 413.9988098144531 1041.233642578125 2096.080078125 3075.3828125
The predicted class is (413, 1041, 2096, 3075)

image 1/1 /content/1000000089.jpg: 640x480 1 ruku, 9.8ms
Speed: 4.5ms preprocess, 9.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.], device='cuda:0')
conf: tensor([0.8187], device='cuda:0')
data: tensor([[4.1400e+02, 1.0412e+03, 2.0961e+03, 3.0754e+03, 8.1874e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (3264, 2448)
shape: torch.Size([1, 6])
xywh: tensor([[1255.0394, 2058.3081, 1682.0813, 2034.1492]], device='cuda:0')
xywhn: tensor([[0.5127, 0.6306, 0.6871, 0.6232]], device='cuda:0')
xyxy: tensor([[ 413.9988, 1041.2336, 2096.0801, 3075.3828]], device='cuda:0'

INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:09:53] "POST /Takbeer HTTP/1.1" 200 -



image 1/1 /content/1000000089.jpg: 640x480 1 ruku, 55.1ms
Speed: 4.2ms preprocess, 55.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)
Bounding box coordinates (x1, y1, x2, y2): 413.9988098144531 1041.233642578125 2096.080078125 3075.3828125
The predicted class is (413, 1041, 2096, 3075)

image 1/1 /content/1000000089.jpg: 640x480 1 ruku, 13.3ms
Speed: 9.6ms preprocess, 13.3ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 480)
5.5854333028576235
Done


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 08:10:37] "POST /Ruku HTTP/1.1" 200 -


In [ ]:
from google.colab import drive
drive.mount('/content/drive')